In [4]:
using CPUTime
using DelimitedFiles
using Base
using LightGraphs, SimpleWeightedGraphs


In [5]:
function readDataFromFile(filename="in7.txt")
    open(filename, "r") do f 
        n, m = split(readline(f))
        n = parse(Int, n)
#         cnt=0
        sources = Vector{Int}()
        destinations = Vector{Int}()
        weights = Vector{Float64}()
        for ln in eachline(f)
#             a,b,c = split(ln)
            a,b,c = readdlm(IOBuffer(ln), Int)
            if(a==0)
                a=n
            end
            if(b==0)
                b=n
            end
#             cnt+=1
#             if(cnt%10000==0)
#                 println(cnt)
#             end
            if(c ==0)
                push!(sources,a)
                push!(destinations,b)
                push!(weights,0.000001)
            else
                push!(sources,a)
                push!(destinations,b)
                push!(weights,c)
            end
        end
        g = SimpleWeightedGraph(sources, destinations, weights)
        return g
    end
end
# for in_n_10e5_m_5e6.in, generate the file from graph_generator.ipynb
g = readDataFromFile("in_n_10e5_m_5e6.in")
# g = readDataFromFile("in7.txt")

{100000, 5000000} undirected simple Int64 graph with Float64 weights

In [13]:
include("boruvka_multithread_lightgraph.jl")

contractVertex (generic function with 1 method)

In [14]:
@time @CPUtime (mst, weight) = boruvka_mst_multithread(g)
println(length(mst))
println(weight)

Max iteration: 18
elapsed CPU time: 2.617068 seconds
  1.063898 seconds (395.16 k allocations: 48.697 MiB)
99999
6018132840
